In [1]:
import pandas as pd
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
import os
import plotly.express as px
import numpy as np
import statsmodels.api as sm
from sklearn import linear_model, kernel_ridge, ensemble, preprocessing, multioutput, svm
from sklearn.decomposition import FastICA, PCA
import lightgbm as lgb
# import warnings filter
from warnings import simplefilter

# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

_LOGTRANS_EPS = 1e-4

results_dir = "/home/gh/postgres/cmudb/tscout/"

#### Load Data

In [2]:
ou_name_to_df = dict()

for filename in os.listdir(results_dir):
    if not filename.endswith("csv"):
        continue

    filepath = os.path.join(results_dir, filename)
    ou_name = filename.split(".")[0]

    if os.stat(filepath).st_size > 0:
        ou_name_to_df[ou_name] = pd.read_csv(filepath)

In [3]:
for (ou_name, ou_df) in ou_name_to_df.items():
    print(f"OU Name: {ou_name}, ou_df shape: {ou_df.shape}")

OU Name: nodeIndexscan_ExecIndexScan, ou_df shape: (578441, 86)
OU Name: nodeLockRows_ExecLockRows, ou_df shape: (447853, 75)
OU Name: nodeValuesscan_ExecValuesScan, ou_df shape: (434290, 71)
OU Name: nodeResult_ExecResult, ou_df shape: (4594, 66)
OU Name: nodeModifyTable_ExecModifyTable, ou_df shape: (19987, 89)


# Pre-processing

#### Get IndexScan OU Data

In [4]:
target_ou = 'nodeIndexscan_ExecIndexScan'
df = ou_name_to_df[target_ou]

#### Remove constant columns

In [5]:
cols_to_remove = []
for col in df.columns:
    if df[col].nunique() == 1: 
            cols_to_remove.append(col)

df = df.drop(cols_to_remove, axis=1)
# print(f"Dropped zero-variance columns: {cols_to_remove}")
print(f"Num Remaining: {len(df.columns)}, Num Removed {len(cols_to_remove)}")

Num Remaining: 35, Num Removed 51


# Summary Statistics

In [6]:
df.describe()

,query_id,IndexScanState_ss_ps_plan,IndexScanState_ss_ps_state,IndexScanState_ss_ps_qual,IndexScanState_ss_ps_ps_ResultTupleDesc,IndexScanState_ss_ps_ps_ResultTupleSlot,IndexScanState_ss_ps_ps_ExprContext,IndexScanState_ss_ps_ps_ProjInfo,IndexScanState_ss_ps_scandesc,IndexScanState_ss_ss_currentRelation,...,Plan_extParam,Plan_allParam,start_time,end_time,cpu_id,cpu_cycles,instructions,cache_references,cache_misses,elapsed_us
count,5.784410e+05,5.784410e+05,5.784410e+05,5.784410e+05,5.784410e+05,5.784410e+05,5.784410e+05,5.784410e+05,5.784410e+05,5.784410e+05,...,5.784410e+05,5.784410e+05,5.784410e+05,5.784410e+05,578441.000000,5.784410e+05,5.784410e+05,5.784410e+05,5.784410e+05,578441.000000
mean,8.912548e+18,9.396078e+13,9.396078e+13,4.517725e+12,9.396078e+13,9.396078e+13,9.396078e+13,9.396078e+13,1.401624e+14,1.401624e+14,...,9.396078e+13,9.396078e+13,5.632572e+11,5.632572e+11,16.379193,1.658303e+15,5.580829e+15,3.189045e+14,1.626413e+15,19.013450
std,5.165418e+18,6.809070e+05,3.809693e+05,2.010174e+13,3.809749e+05,3.809750e+05,3.809691e+05,3.809751e+05,3.930062e+04,3.930505e+04,...,3.698818e+05,3.698819e+05,1.063828e+07,1.063827e+07,8.300940,1.748931e+17,3.208070e+17,7.669849e+16,1.732034e+17,58.777027
min,1.305452e+18,9.396078e+13,9.396078e+13,0.000000e+00,9.396078e+13,9.396078e+13,9.396078e+13,9.396078e+13,1.401624e+14,1.401624e+14,...,9.396078e+13,9.396078e+13,5.632363e+11,5.632363e+11,8.000000,8.012000e+03,1.821000e+03,1.298000e+03,1.500000e+01,1.000000
25%,6.654292e+18,9.396078e+13,9.396078e+13,0.000000e+00,9.396078e+13,9.396078e+13,9.396078e+13,9.396078e+13,1.401624e+14,1.401624e+14,...,9.396078e+13,9.396078e+13,5.632496e+11,5.632496e+11,9.000000,1.340700e+04,2.440800e+04,1.640000e+03,1.380000e+02,2.000000
50%,6.654292e+18,9.396078e+13,9.396078e+13,0.000000e+00,9.396078e+13,9.396078e+13,9.396078e+13,9.396078e+13,1.401624e+14,1.401624e+14,...,9.396078e+13,9.396078e+13,5.632584e+11,5.632584e+11,12.000000,1.564400e+04,2.770800e+04,1.782000e+03,1.880000e+02,3.000000
75%,1.570483e+19,9.396078e+13,9.396078e+13,0.000000e+00,9.396078e+13,9.396078e+13,9.396078e+13,9.396078e+13,1.401624e+14,1.401624e+14,...,9.396078e+13,9.396078e+13,5.632660e+11,5.632660e+11,26.000000,6.691900e+04,2.389920e+05,2.618000e+03,3.280000e+02,15.000000
max,1.570483e+19,9.396078e+13,9.396078e+13,9.396078e+13,9.396078e+13,9.396078e+13,9.396078e+13,9.396078e+13,1.401624e+14,1.401624e+14,...,9.396078e+13,9.396078e+13,5.632738e+11,5.632738e+11,31.000000,1.844674e+19,1.844674e+19,1.844674e+19,1.844674e+19,816.000000


# Visualizations

In [7]:
target_col = 'elapsed_us'

# for col in df.columns:
#     fig = px.scatter(x=df[col], y=df[target_col])
#     fig.show()

#### Correlation Heatmap

In [8]:
# df_corr = df.corr()
# plt.figure(figsize=(10,7))
# sns.heatmap(df_corr, cmap='GnBu_r')
# plt.show()

# Feature Analysis (PCA)

In [9]:
pca = PCA(ncomponents=None)
pca.fit(df)
pca.explained_variance_ratio_

TypeError: __init__() got an unexpected keyword argument 'ncomponents'

#### ICA to look at signal relationships

In [ ]:
# ica = FastICA(n_components=7)
# ica.fit(df)
# print(ica.components_.shape)

# Modeling

In [ ]:
X = df.drop('elapsed_us', axis=1)
Y = df['elapsed_us']

#### Actual Statistics

In [ ]:
results = sm.OLS(Y, X).fit()
results.summary()

In [ ]:
X = df.drop('elapsed_us', axis=1)
Y = df['elapsed_us']

#### Sklearn OLS

In [ ]:
lin_reg = sklearn.linear_model.LinearRegression()
fit = lin_reg.fit(X, Y)
fit

In [ ]:
def _get_base_ml_model(method):
    regressor = None
    if method == 'lr':
        regressor = linear_model.LinearRegression()
    if method == 'huber':
        regressor = linear_model.HuberRegressor(max_iter=50)
        regressor = multioutput.MultiOutputRegressor(regressor)
    if method == 'svr':
        regressor = svm.LinearSVR()
        regressor = multioutput.MultiOutputRegressor(regressor)
    if method == 'kr':
        regressor = kernel_ridge.KernelRidge(kernel='rbf')
    if method == 'rf':
        regressor = ensemble.RandomForestRegressor(n_estimators=50, n_jobs=8)
    if method == 'gbm':
        regressor = lgb.LGBMRegressor(max_depth=20, num_leaves=1000, n_estimators=100, min_child_samples=5,
                                      random_state=42)
        regressor = multioutput.MultiOutputRegressor(regressor)
    if method == 'nn':
        regressor = neural_network.MLPRegressor(hidden_layer_sizes=(25, 25), early_stopping=True,
                                                max_iter=1000000, alpha=5)

    return regressor


class Model:
    """
    The class that wraps around standard ML libraries.
    With the implementation for different normalization handlings
    """

    def __init__(self, method, normalize=True, log_transform=True, y_transformer=None, x_transformer=None):
        """
        :param method: which ML method to use
        :param normalize: whether to perform standard normalization on data (both x and y)
        :param log_transform: whether to perform log transformation on data (both x and y)
        :param y_transformer: the customized data transformer for output (a pair of functions with the first for
               training and second for predict)
        :param x_transformer: the customized data transformer for input
        """
        self._base_model = _get_base_ml_model(method)
        self._normalize = normalize
        self._log_transform = log_transform
        self._xscaler = preprocessing.StandardScaler()
        self._yscaler = preprocessing.StandardScaler()
        self._y_transformer = y_transformer
        self._x_transformer = x_transformer

    def train(self, x, y):
        if self._y_transformer is not None:
            y = self._y_transformer[0](x, y)

        if self._x_transformer is not None:
            x = self._x_transformer(x)

        if self._log_transform:
            x = np.log(x + _LOGTRANS_EPS)
            y = np.log(y + _LOGTRANS_EPS)

        if self._normalize:
            x = self._xscaler.fit_transform(x)
            y = self._yscaler.fit_transform(y)

        self._base_model.fit(x, y)

    def predict(self, x):
        original_x = x

        if self._x_transformer is not None:
            x = self._x_transformer(x)

        # transform the features
        if self._log_transform:
            x = np.log(x + _LOGTRANS_EPS)
        if self._normalize:
            x = self._xscaler.transform(x)

        # make prediction
        y = self._base_model.predict(x)

        # transform the y back
        if self._normalize:
            y = self._yscaler.inverse_transform(y)
        if self._log_transform:
            y = np.exp(y) - _LOGTRANS_EPS
            y = np.clip(y, 0, None)

        if self._y_transformer is not None:
            y = self._y_transformer[1](original_x, y)

        return y